In [1]:
# Run this code block just once after the kernel starts
# to change current directory to root

import sys
import os

sys.path.append("..")
os.chdir("..")
print(os.getcwd())

/home/student/bev-project


In [2]:
from dataset import NuScenesDataset
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from nuscenes_utilities import NUSCENES_CLASS_NAMES

In [3]:
from configs.config_utilities import load_config
config = load_config("configs/configs.yml")

In [4]:
print(config.train_tokens)
print(config.val_tokens)

['f1680ea9126e468ca407f7284efc980c', 'bf4b00d2af9344c3b3f444206f75cf14', '73fffafd944c4c42a32cfcaeeafc4944', '9d8d518a34ff462d8d6a8fb665d5edd4', 'd209059d97174d28a279f467e9cf89f0', '19f4244aa37b42a49a005b8ea6d53953', 'c33b84588ead4a609f195d2c4fb56801', '731b03604b4849f0aa646d0b46571451', 'f331f492972540ab8e843d484ac6919e', '834ce773cfa04be8b56d9614ed1dcf7f', '50e02dd80f9444a194abe8acae4bb211', 'd507acb08ec94f00b54bd650bb4b1c52', 'b4f9181a80034d22972433e4d1dd9f9e', 'b04ad7e3c86d47fc98dba25595b2327e', '76542abab799469cbfb7b293c83192aa', 'beea8d83b337481799c36c6561ec3de6', '6830e208ec3b444f878c8f5c11135dc6', 'deb67484315f4ba8b56ea8f03ef3d444', 'f059772e809443b787af4bb7c60c094a', '9b6ce88572b947b3b8e141c8bfe96f5b', '3cd43f6baa4e444999eef6cb1c89416a', 'e26810c28df44cefa069778865bd832c', '881817a81bab4d8488464ac42db89f5f', 'e30d5ecf0fd6418089112b4e4811737c', 'eeed80879b3d4580804a0da6bf9c84e2', 'b732ab7ce371484b93ba0c2d2b2f9ae7', 'fc4cba53bd2c4923b17da19b2e6e44fa', '7b6c123b090340358e642a70c6

In [5]:
dataset = NuScenesDataset(
    nuscenes_dir=config.nuscenes_dir,
    nuscenes_version=config.nuscenes_version,
    label_dir=config.label_dir,
    sample_tokens=config.train_tokens + config.val_tokens,
    image_size=(200, 112),
    flatten_labels=(config.method_type == "multiclass"),
)
dataset_loader = DataLoader(
    dataset,
    batch_size=10,
    num_workers=2,
    pin_memory=True,
    shuffle=True,
)

--------------------------------------------------
Loading NuScenes version v1.0-mini ...
--------------------------------------------------


In [6]:
for images, labels, masks, calibs in dataset_loader:
    print(labels.shape)

torch.Size([10, 14, 196, 200])
torch.Size([10, 14, 196, 200])
torch.Size([10, 14, 196, 200])
torch.Size([10, 14, 196, 200])
torch.Size([1, 14, 196, 200])


In [7]:
NUSCENES_CLASS_NAMES

['drivable_area',
 'ped_crossing',
 'walkway',
 'carpark',
 'car',
 'truck',
 'bus',
 'trailer',
 'construction_vehicle',
 'pedestrian',
 'motorcycle',
 'bicycle',
 'traffic_cone',
 'barrier']

In [8]:
B, C, H, W = labels.shape
print(labels.permute(1,0,2,3).shape)
print(labels.permute(1,0,2,3).flatten(1, 3).shape)
print(labels.permute(1,0,2,3).flatten(1, 3).sum(dim=1).shape)
print(labels.permute(1,0,2,3).flatten(1, 3).sum(dim=1))

torch.Size([14, 1, 196, 200])
torch.Size([14, 39200])
torch.Size([14])
tensor([15215,  3033,  3647,     0,   364,   419,     0,     0,     0,    18,
            0,     0,     0,     0])


In [9]:
pixel_counts = None
for images, labels, masks, calibs in dataset_loader:
    pixel_count = labels.permute(1,0,2,3).flatten(1, 3).sum(dim=1)
    if pixel_counts is None:
        pixel_counts = pixel_count
    else:
        pixel_counts = pixel_counts + pixel_count

print(pixel_counts)

tensor([633525, 128399, 144515,    911,  21080,   9054,  15798,      0,   1152,
           624,      0,    770,    114,      0])


In [10]:
pixel_counts / pixel_counts.sum()

tensor([6.6272e-01, 1.3432e-01, 1.5118e-01, 9.5299e-04, 2.2052e-02, 9.4713e-03,
        1.6526e-02, 0.0000e+00, 1.2051e-03, 6.5276e-04, 0.0000e+00, 8.0549e-04,
        1.1925e-04, 0.0000e+00])

In [11]:
pixel_counts_percentage = pixel_counts / pixel_counts.sum()

for classname, percetage in zip(NUSCENES_CLASS_NAMES ,pixel_counts_percentage):
    print(f"{classname} : {percetage}")

drivable_area : 0.6627232432365417
ped_crossing : 0.13431672751903534
walkway : 0.15117548406124115
carpark : 0.0009529867093078792
car : 0.022051546722650528
truck : 0.009471286088228226
bus : 0.016526106745004654
trailer : 0.0
construction_vehicle : 0.0012050940422341228
pedestrian : 0.0006527592777274549
motorcycle : 0.0
bicycle : 0.0008054882055148482
traffic_cone : 0.00011925409489776939
barrier : 0.0
